In [1]:
import pandas as pd
dataset = pd.read_csv("./healthcare-dataset-stroke-data.csv")

In [2]:
dataset

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [3]:
print(dataset['smoking_status'].value_counts())

never smoked       1892
Unknown            1544
formerly smoked     885
smokes              789
Name: smoking_status, dtype: int64


In [7]:
dataset["smoking_status"] = dataset["smoking_status"].apply(lambda x: x.replace("formerly smoked","smokes"))
dataset["bmi"] = dataset["bmi"].fillna(dataset["bmi"].mean())

In [8]:
# drop gender == other since it only has 1 row
indexToDrop = dataset[dataset["gender"] == "Other"].index
dataset.drop(indexToDrop,inplace=True)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
to_encode = ["gender","work_type","Residence_type","ever_married","smoking_status"]
def encode(colName):
    newName = colName + "_encoded"
    dataset[newName] = le.fit_transform(dataset[colName])
    return dataset

for x in to_encode:
    encode(x)

dataset

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,gender_encoded,work_type_encoded,Residence_type_encoded,ever_married_encoded,smoking_status_encoded
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,smokes,1,1,2,1,1,2
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1,0,3,0,1,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1,1,2,0,1,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1,0,2,1,1,2
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1,0,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,28.893237,never smoked,0,0,2,1,1,1
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.000000,never smoked,0,0,3,1,1,1
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.600000,never smoked,0,0,3,0,1,1
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.600000,smokes,0,1,2,0,1,2


In [10]:
from sklearn.neighbors import KNeighborsClassifier
X = dataset[["age","hypertension","heart_disease","avg_glucose_level","bmi","gender_encoded","work_type_encoded","Residence_type_encoded","ever_married_encoded","smoking_status_encoded"]]
y = dataset["stroke"]

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
knn = KNeighborsClassifier(leaf_size=10, n_neighbors=5)
knn.fit(X_train,y_train)
knnPred = knn.predict(X_test)

print(classification_report(y_test,knnPred))
print(confusion_matrix(y_test,knnPred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      1462
           1       0.18      0.03      0.05        71

    accuracy                           0.95      1533
   macro avg       0.57      0.51      0.51      1533
weighted avg       0.92      0.95      0.93      1533

[[1453    9]
 [  69    2]]
